In [1]:
# !pip install -U torch torchvision

In [2]:
# !pip install -U git+https://github.com/openai/CLIP.git

In [5]:
import torch
import clip
from PIL import Image
import os
import numpy as np
import pandas as pd
import csv
import pycountry

In [7]:
# Load the pre-trained CLIP model and the image
model, preprocess = clip.load('ViT-B/32')

In [8]:
def get_clip_text_features(model, preprocess, text_list):
    
    feature=[] 
    
    for text in text_list:
        text_input = clip.tokenize([text])
        
        # Move the inputs to GPU if available
        device = "cuda" if torch.cuda.is_available() else "cpu"
        text_input = text_input.to(device)
        model = model.to(device)
        
        with torch.no_grad():
            text_features = model.encode_text(text_input)
        
        # Normalize the features
        text_features = text_features / text_features.norm(dim=-1, keepdim=True)
        feature.append(text_features)
    
    return feature

In [6]:
all_countries = list(pycountry.countries)
country_names = [country.name for country in all_countries]

In [9]:
features = get_clip_text_features(model, preprocess,country_names)

In [13]:
# storing the list of tensor the features in csv file
torch.save(features, 'model/eval_country_text_features.pt')

In [14]:
# Save the list of strings to a text file
with open('model/eval_country_text_country_list.txt', 'w') as file:
    for string in country_names:
        file.write(string + '\n')